<h1><u>Introducció</u></h1>

L'1 de Desembre de 2019 vam anar amb uns amics a fer una excursió i collir bolets pel Berguedà. Durant el trajecte en cotxe de camí cap allà vam parlar de molts temes diferents, entre ells el *deep learning*. Coincidint que anàvem a buscar bolets, un amic meu i jo vam començar a discutir sobre la possibilitat de fer servir una xarxa neuronal per detectar l'espècie a la qual pertany un bolet a partir d'una fotografia. Jo deia que era relativament fàcil de fer i ell deia que no, que els bolets s'assemblen molt els uns als altres i que sovint es sobreestimaven les capacitats del *deep learning*. No entraré a discutir la veracitat d'aquesta frase, però sí que em veia capaç de ser jo mateix qui programés i entrenés la xarxa neuronal en qüestió. Finalment li vaig dir al meu amic: "Faré una 'app' per detectar espècies de bolets i l'anomenaré *Joboleté*", fruit de la combinació entre la paraula 'bolet' i el seu cognom. Entre els amics ens ho vam prendre inicialment com una broma, però jo vaig pensar que seria un exercici interessant intentar desenvolupar la 'app' en un futur, o almenys entrenar una xarxa neuronal que assolís un grau satisfactori de fiabilitat a l'hora de predir espècies de bolets.

Durant el confinament de la primavera del 2020 vaig trobar temps per començar a mirar-me una mica el projecte. Segurament la millor opció hagués sigut contactar experts en micologia des de l'inici, però primer vaig decidir veure fins on podia arribar pel meu compte. Abans de començar a mirar-me res, però, necessitava un esbós mental del resultat que volia obtenir. Resumint-ho de manera informal: "Una 'app' amb la qual puguis fer-li una foto a un bolet i et digui de quina espècie de bolet es tracta, si és comestible o no i, en cas que ho sigui, suggerir alguna recepta" (aquesta última idea va sorgir mentre caminàvem pel bosc i va ser suggerència d'un dels meus amics). Per no complicar-ho massa, vaig decidir considerar només les espècies de bolets que es troben a Catalunya.

Així doncs, vaig dividir el projecte en **5 fases** diferents:
1. Trobar els noms de les espècies de bolets presents a Catalunya
2. Descarregar-me totes les imatges disponibles de cada espècie de bolet
3. Processar les imatges, cosa que inclou netegar i possiblement etiquetar i utilitzar tècniques de *data augmentation* per obtenir el millor *dataset* possible per entrenar la xarxa neuronal
4. Dissenyar i entrenar xarxes neuronals amb el *training set* i quedar-me amb la que obtingui millors resultats al *test set*
5. Desenvolupar la resta de la 'app'

La primavera de l'any passat vaig arribar gairebé al final de la fase 2. Aleshores vaig escriure el codi de qualsevol manera per tal d'aconseguir ràpidament les imatges de bolets, que era el que realment m'interessava per poder entrenar la meva xarxa neuronal. Vaig arribar a descarregar-me algunes imatges (tot i que moltes menys de les que volia) i vaig estar experimentant provant d'entrenar una ResNet18 i una VGG11 al Google Colab sense èxit. Com que en aquell moment no podia dedicar-li el temps suficient per tirar endavant el projecte, vaig optar per posposar-lo.

Ara que torno a tenir temps he decidit tornar a mirar-me'l i posar-hi una mica d'ordre. He de reconèixer que al principi va costar recordar-me'n de com funcionaven exactament algunes de les coses que vaig programar fa temps. Tot i així, he intentat netejar, modularitzar i comentar el codi en la mesura que ha estat possible, deixant les coses que he considerat que no valia la pena canviar. El més probable és que no sigui ara quan acabi el projecte ni tan sols en un futur proper, però tenia pendent organitzar i comentar la feina que vaig fer fa temps amb l'objectiu d'aconseguir com a mínim escriure un programa que sigui capaç de descarregar-me les imatges de bolets que necessito per entrenar la meva xarxa neuronal. Dit això, començaré explicant la fase 1: Com trobar els noms de les espècies de bolets presents a Catalunya. 

<h3>Buscant les espècies de bolets de Catalunya</h3>

Vaig començar buscant només espècies de bolets comestibles, ja que vaig pensar que tenir-los separats dels verinosos em facilitaria la feina més endavant. Una forma ràpida de trobar els noms de bolets és cercant "bolets comestibles" o "bolets comestibles Catalunya" a Google. Hi ha diverses pàgines de bolets on surten les espècies més freqüents a Catalunya. Segurament anant un per un i apuntat els noms de cada pàgina manualment hagués acabat abans, però vaig pensar que seria més entretingut fer un programa que trobés els noms dels bolets automàticament a partir de les *URL* de cada pàgina. Com que no havia fet mai *web scrapping*, vaig buscar quines llibreries de Python s'utilitzen freqüentment per aquest tipus de tasques. En vaig trobar tres que són relativament senzilles d'utilitzar: *Requests*, *Beautiful Soup* i *Selenium*. A continuació explico el procediment per obtenir els noms dels bolets en català i en llatí.

Comencem important les llibreries necessàries per fer *web scrapping* i altres llibreries i mòduls que també utilitzarem:

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import scrap
from utils import save_content_to_file, read_json_file

Per obtenir l'HTML d'una pàgina web fem servir *requests.get(url)*, el qual ens retorna un objecte Response. Si l'status de l'objecte Response és 200 vol dir que la nostra cerca ha estat existosa. Per contra, si l'status és de la forma 4XX vol dir que hi ha hagut un problema amb el client, i si l'status és de la forma 5XX vol dir que hi ha hagut un problema amb el servidor.

In [2]:
response = requests.get('http://mas.regio7.cat/bolets/especies.html')
print(type(response))
print(response.status_code)

<class 'requests.models.Response'>
200


Tot bé. Amb l'objecte Response podem obtenir el text de l'HTML amb *response.text*. El següent pas és fer el que en anglès s'anomena *parsing*, que consisteix a fer el text sigui més llegible i ens permet accedir a la informació que ens interessa més fàcilment. Per fer-ho utilitzem Beautiful Soup:

In [3]:
soup = BeautifulSoup(response.text, parser = 'lxml')

Per tal de trobar els noms dels bolets necessitem saber les etiquetes dels elements de l'HTML que els contenen. Una manera ràpida de fer-ho és anant a la pagina web en qüestió, fent click amb el botó secundari a qualsevol lloc de la pàgina i clicant *inspect* (o *inspeccionar*) tal i com mostra la següent figura (esquerra). Al navegador ens apareix un panell DOM (sigles per Document Object Model en anglès) i en aquest cas clicarem a la icona enquadrada en vermell tal i com mostra la figura de la dreta. 

![Figure 1: Inspect web example](Figures/inspect_web.png "Inspect web example")

Si passem el cursor per sobre el nom del bolet (o de qualsevol element de la pàgina) se'ns apareix al panell l'etiqueta exacta on es troba l'element, juntament amb totes les etiquetes 'pare' que queden per sobre. En aquest cas tots els noms de bolets es troben a la taula amb *class="blog"* i etiqueta *td* amb *class="contentheading"*:

![Figure 2: Find HTML tag](Figures/find_names.png "Find HTML tag")

Un cop sabem les etiquetes dels elements que volem buscar podem trobar-los fent servir *soup.find* i *soup.findAll* o *soup.find_all*:

In [4]:
text = soup.find('table', {'class': 'blog'}).findAll('td', {'class': 'contentheading'})
names = [name.text.replace('\n','').replace('\t','') for name in text]
print(names)

['Apagallum', 'Camagroc', 'Fredolic', 'Llora', 'Múrgola', 'Rovelló', 'Bolet de tinta', 'Camperol', 'Llenega blanca', 'Marçot', 'Pinetell', 'Trompeta', 'Cama de perdiu', 'Cep', 'Llenega negra', 'Moixernó', 'Reig', 'Cama-sec', 'Escarlet', 'Llengua de Bou', 'Molleric', 'Rossinyol']


La funció *findAll* retorna un objecte ResultSet sobre el qual podem iterar. Podem obtenir el contingut de cada un dels elements fent servir *.text*, que ens retorna una string. En algunes pàgines webs aquesta string és exactament el nom del bolet que estem buscant, però en d'altres s'ha de netejar el text. En aquest cas hem de treure els '\n' i els '\t'. La descripció exacta de què fan les funcions *find* i *findAll* la podem trobar a la documentació de Beautiful Soup [1]. 

A *scrap.py* he definit vàries funcions que agrupen alguns d'aquests passos per simplificar una mica el codi, però la funcionalitat és essencialment la mateixa. La funció *find_items* és equivalent a la primera línia de codi de la cel·la anterior i la funció *find_names* és equivalent a la segona. Ambdós funcions admeten certa flexibilitat, cosa que ens permet *scrapejar* webs que utilitzen etiquetes diferents utilitzant el mateix codi.

L'arxiu *url comestibles* a la carpeta *Noms* conté un Pandas *dataframe* amb tota la informació necessària per obtenir els noms dels bolets de 4 pàgines web diferents. El *dataframe* està 'serialitzat' amb Pickle, així que l'obrim de la seguent manera:

In [5]:
df = pd.read_pickle('Noms/url_comestibles')

I en podem veure el contingut:

In [6]:
df

,url,comestible,tag id,grafia,tag especific,replace
0,https://ca.wikipedia.org/wiki/Categoria:Bolets...,Si,"(div, {'class': 'mw-category'})",m,a,()
1,http://mas.regio7.cat/bolets/especies.html,Si,"(table, {'class': 'blog'})",M,"(td, {'class': 'contentheading'})","(\n, \t)"
2,https://web.gencat.cat/ca/actualitat/reportatg...,Si,"(div, {'class': 'llistat_enllacos_filet especi...",m,a,()
3,https://bolets.info/llistat-bolets/bolets-bons/,Si,"(div, {'id': 'main-area'})",m,[h2],()


Amb aquesta informació podem trobar els noms dels bolets de cada una de les pàgines i agrupar-los fàcilment:

In [7]:
bolets_comestibles = []

for idx in df.index:
    soup = BeautifulSoup(requests.get(df['url'][idx]).text, 'lxml')
    text = scrap.find_items(soup, df['tag id'][idx], df['tag especific'][idx], case=df['grafia'][idx])
    names = scrap.get_names(text, replacements=df['replace'][idx])
    bolets_comestibles = list(set(bolets_comestibles) | set(names))
    
print(bolets_comestibles[:10])
print('')
print(len(bolets_comestibles))

['Bromosa', 'Lleterola roja', 'Moraga', 'Rovelló', 'Molleric', 'Cualbra morada', 'Llenega de muntanya', 'Carlet', 'Mataparent de cama roja', 'Armillaria luteobubalina']

137


Com podem veure hem trobat 137 instàncies que coincideixen amb les etiquetes que hem buscat. Aquestes instàncies contenen els noms dels bolets en català i són les que farem servir per trobar els noms en llatí. Aquesta llista no és la definitiva, ja que s'han de corregir algunes coses com per exemple instàncies que contenen strings com 'gastronomia de bolets' que s'han de treure. A més a més, hi ha algunes noms de bolets que en català que contenen caràcters com la *ç* i accents oberts que causen problemes a l'hora d'utilitzar *requests* per cercar els noms en llatí. La primera vegada que vaig escriure el codi vaig decidir fer una llista amb aquests noms i les strings que no són de bolets. Aquesta llista la podem trobar al directori *Noms/excepcions_comestibles_catala.txt* i els corresponents noms en llatí a *Noms/excepcions_comestibles_llati.txt*. Des que he arreglat el codi alguns d'aquests noms ja no causen els mateixos problemes, però d'altres si. Al final he optat per deixar la llista d'excepcions tal i com estava, ja que aquests noms i els corresponents noms en llatí els afegirem igualment a la llista final. 

Per trobar els noms en llatí fem servir la funció *get_latin_names*, que utilitza *requests* i *Beautiful Soup* per buscar cada nom en català a google i trobar el nom en llatí a la wikipedia.

In [8]:
bolets_comestibles += ['Bolet de tinta','Mollerons','Cogomella']

excepcions_catala = read_json_file('Noms/excepcions_comestibles_catala.txt', 'r')
excepcions_llati = read_json_file('Noms/excepcions_comestibles_llati.txt', 'r')

bolets_comestibles = list(set(bolets_comestibles).difference(set(excepcions_catala)))

# afegim la paraula 'bolet' al final de cada nom per facilitar la cerca a google. Per exemple, els bolets
# 'Camperol' o 'Coliflor' són més fàcils de trobar si cerquem 'Camperol bolet' o 'Colifor bolet'. També es
# podria afegir directament dins la funció get_latin_names
cerca_bolets = [bolet + ' bolet' for bolet in bolets_comestibles]

# aquests noms els traiem definitivament
diferencies = ['Tòfona','Cigró','Gastronomia dels bolets','Llenega','Terfeziàcies']

excepcions_catala = list(set(excepcions_catala).difference(set(diferencies)))

# busquem els noms en llatí
noms_llati = scrap.get_latin_names(cerca_bolets)

Cerca de noms en llatí completada


Finalment hi afegim les excepcions que havíem retirat anteriorment i ho guardem tot en un diccionari, on les *keys* són els noms en llatí i els *values* són els noms en català. D'aquesta manera s'eliminaran automàticament totes les *keys* que estiguin repetides que corresponen al mateix bolet, ja que en llatí cada espècie s'anomena d'una única manera encara que en català es pugui anomenar de maneres diferents.

In [9]:
bolets_comestibles += excepcions_catala
noms_llati += excepcions_llati

bolets = dict(zip(noms_llati, bolets_comestibles))

# guardem el diccionari final com una string en un arxiu .txt
save_content_to_file(str(bolets),"Noms/bolets_comestibles.txt","w")

Aquesta és la nostra llista definitiva d'espècies de bolets comestibles de Catalunya:

In [10]:
print(bolets)

{'Clitocybe nebularis': 'Bromosa', 'Lactarius volemus': 'Lleterola roja', 'Helvella spadicea': 'Moraga', 'Lactarius sanguifluus': 'Rovelló', 'Suillus granulatus': 'Molleric granellut', 'Russula cyanoxantha': 'Cualbra morada', 'Hygrophorus olivaceoalbus': 'Llenega de muntanya', 'Hygrophorus russula': 'Escarlet', 'Armillaria luteobubalina': 'Armillaria luteobubalina', 'Hydnum repandum': 'Llengua de bou (bolet)', 'Chlorophyllum rhacodes': 'Apagallum', 'Laccaria laccata': 'Pentinella rosada', 'Agaricus campestris': 'Tòfona de Borch', 'Rhizopogon roseolus': 'Fetjó', 'Agaricus silvicola': 'Bola-de-neu de bosc', 'Boletus edulis': 'Boletus pinicola', 'Marasmius oreades': 'Cama-sec', 'Tricholoma terreum': 'Fredolic', 'Lactarius vellereus': 'Pebrassa vellutada', 'Hygrophorus latitabundus': 'Llenega negra', 'Amanita caesarea': 'Ou de Reig', 'Laetiporus sulphureus': 'Gírgola groga', 'Chroogomphus rutilus': 'Bec de perdiu', 'Hygrophorus marzuolus': 'Tricoloma fosc', 'Helvella lacunosa': 'Orella de 

Tot el que hem vist en aquest *notebook* es pot dur a terme simplement executant la funció *main* de l'script *scrap.py*. L'script en si no té molta utilitat més enllà de poder reutilitzar alguna funció, ja que els noms dels bolets comestibles els podem trobar en aquest repositori a *Noms/bolets_comestibles.txt*. Si bé és cert que algunes parts de l'script són millorables i es podrien generalitzar per poder-lo utilitzar per tasques similars, l'objectiu pel qual vaig escriure el codi ja l'he assolit: trobar els noms en català i en llatí de les espècies de bolets comestibles que es troben a Catalunya.

Els noms de les espècies de bolets verinosos a Catalunya els podem trobar a *Noms/bolets_verinosos.txt* amb el mateix format que els bolets comestibles. Aquests noms els vaig buscar a part (abans d'organitzar el codi) fent servir els links que es troben al final d'aquest *notebook*, abans de les referències. Els anomenats 'bolets ambigus' són bolets que si es tracten adequadament són comestibles, però que en determinades situacions poden resultar tòxics. Els podem trobar a *Noms/bolets_ambigus.txt*

Ara que hem arribat al final de la fase 1 podem passar a la fase 2: descarregar-nos les imatges de bolets. Les espècies de bolets que hem trobat no són ni de bon troç totes les que es troben a Catalunya, però sí que probablement representen la gran majoria de bolets que ens trobem quan anem a collir-ne. El codi de la fase 2 el podem trobar a l'arxiu *get_images.py*, i l'explicació detallada del codi estarà disponible a *imatges.ipynb*. Aquest cop espero aconseguir escriure un programa que permeti descarregar-me totes les imatges de bolets que vulgui.

*Girona, 25 de Febrer del 2021*

<h3>Links bolets verinosos:</h3>

- https://www.rac1.cat/societat/20191011/47903298949/bolets-toxics-verinosos-habituals-catalunya-micologia-perill-agencia-catalana-de-seguretat-alimentaria.html
- https://ca.wikipedia.org/wiki/Intoxicaci%C3%B3_per_bolets
- https://web.gencat.cat/ca/actualitat/reportatges/temporada-de-bolets/bolets/bolets-toxics/
- https://www.regio7.cat/bergueda/2017/09/25/bolets-mes-perilloses-que-existeixen/435569.html
- https://bolets.info/llistat-bolets/bolets-sense-valor-culinari-i-toxics/
- https://hermesalud.com/downloads/bolets-toxics---generalitat-de-catalunya.pdf

<h3>Referències</h3>

- [1] https://www.crummy.com/software/BeautifulSoup/bs4/doc/
- [2] https://requests.readthedocs.io/en/master/